In [5]:
import torch
from transformers import BertTokenizer, BertModel, AutoModel
import pandas as pd
# import BERT-base pretrained model
bert = AutoModel.from_pretrained('bert-base-uncased')

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [19]:
# Create a function to tokenize a set of texts
def preprocessing_for_bert(data):
    """Perform required preprocessing steps for pretrained BERT.
    @param    data (np.array): Array of texts to be processed.
    @return   input_ids (torch.Tensor): Tensor of token ids to be fed to a model.
    @return   attention_masks (torch.Tensor): Tensor of indices specifying which
                  tokens should be attended to by the model.
    """
    encoded_sent = tokenizer.encode_plus(
            text=data,  # Preprocess sentence
            add_special_tokens=True,        # Add `[CLS]` and `[SEP]`
            max_length=MAX_LEN,                  # Max length to truncate/pad
            pad_to_max_length=True,         # Pad sentence to max length
            #return_tensors='pt',           # Return PyTorch tensor
            return_attention_mask=True      # Return attention mask
            )

    input_ids = torch.tensor(encoded_sent.get('input_ids')).unsqueeze(0)
    attention_masks = torch.tensor(encoded_sent.get('attention_mask')).unsqueeze(0)

    return input_ids, attention_masks

In [20]:
from transformers import AutoTokenizer
def find_BERT_embeddings(texts,qlen = 512):
    
    input_ids,attention_masks = preprocessing_for_bert(texts)
    
    with torch.no_grad():
        x1 = bert(input_ids=input_ids,
                            attention_mask=attention_masks)
        
        out = x1[0][:, 0, :].to("cpu")
        del input_ids,attention_masks,x1
    return out

In [6]:
df = pd.read_csv("spotify_songs.csv")
df.head()

,track_id,track_name,track_artist,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,playlist_genre,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,6f807x0ima9a1j3VPbc7VN,I Don't Care (with Justin Bieber) - Loud Luxur...,Ed Sheeran,66,2oCs0DGTsRO98Gh5ZSl2Cx,I Don't Care (with Justin Bieber) [Loud Luxury...,2019-06-14,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,6,-2.634,1,0.0583,0.1020,0.000000,0.0653,0.518,122.036,194754
1,0r7CVbZTWZgbTCYdfa2P31,Memories - Dillon Francis Remix,Maroon 5,67,63rPSO264uRjW1X5E6cWv6,Memories (Dillon Francis Remix),2019-12-13,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,11,-4.969,1,0.0373,0.0724,0.004210,0.3570,0.693,99.972,162600
2,1z1Hg7Vb0AhHDiEmnDE79l,All the Time - Don Diablo Remix,Zara Larsson,70,1HoSmj2eLcsrR0vE9gThr4,All the Time (Don Diablo Remix),2019-07-05,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,1,-3.432,0,0.0742,0.0794,0.000023,0.1100,0.613,124.008,176616
3,75FpbthrwQmzHlBJLuGdC7,Call You Mine - Keanu Silva Remix,The Chainsmokers,60,1nqYsOef1yKKuGOVchbsk6,Call You Mine - The Remixes,2019-07-19,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,7,-3.778,1,0.1020,0.0287,0.000009,0.2040,0.277,121.956,169093
4,1e8PAfcKUYoKkxPhrHqw4x,Someone You Loved - Future Humans Remix,Lewis Capaldi,69,7m7vv9wlQ4i0LFuJiE2zsQ,Someone You Loved (Future Humans Remix),2019-03-05,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,1,-4.672,1,0.0359,0.0803,0.000000,0.0833,0.725,123.976,189052


In [7]:
print(df.columns)

Index(['track_id', 'track_name', 'track_artist', 'track_popularity',
       'track_album_id', 'track_album_name', 'track_album_release_date',
       'playlist_name', 'playlist_id', 'playlist_genre', 'playlist_subgenre',
       'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'duration_ms'],
      dtype='object')


In [13]:
df["text"] = ""
for i in df.index:
    track_name = df["track_name"][i]
    track_artist = df["track_artist"][i]
    track_album_name = df["track_album_name"][i]
    playlist_name = df["playlist_name"][i]
    playlist_genre = df["playlist_genre"][i]
    playlist_subgenre = df["playlist_subgenre"][i]
    df["text"][i] = f"{track_name} is a song by artist {track_artist} from album {track_album_name}. The song is found in playlist {playlist_name} which is of {playlist_genre} genre and {playlist_subgenre} subgenre."

<ipython-input-13-37ad817636d7>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text"][i] = f"{track_name} is a song by artist {track_artist} from album {track_album_name}. The song is found in playlist {playlist_name} which is of {playlist_genre} genre and {playlist_subgenre} subgenre."


In [14]:
df["text"][0]

"I Don't Care (with Justin Bieber) - Loud Luxury Remix is a song by artist Ed Sheeran from album I Don't Care (with Justin Bieber) [Loud Luxury Remix]. The song is found in playlist Pop Remix which is of pop genre and dance pop subgenre."

In [15]:
len(df)

32833

In [27]:
les = [len(text.split(" ")) for text in df["text"]]
max(les)

95

In [28]:
df["BERT_embeddings"] = "" 
MAX_LEN = 98
count = 0
for i in df.index:
    count+=1
    if count % 100 == 0:
        print(count)
    df["BERT_embeddings"][i] = find_BERT_embeddings(df["text"][i]).squeeze()
    

<ipython-input-28-8dfab3cee9cc>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["BERT_embeddings"][i] = find_BERT_embeddings(df["text"][i]).squeeze()


100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
1850

In [25]:
df["BERT_embeddings"][0].squeeze().shape

torch.Size([1, 768])

In [29]:
import pickle
pickle.dump(df,open("Hardik_spotify_df.pickle","wb"))

In [1]:
import pandas as pd
import pickle
checkdf = pickle.load(open("Hardik_spotify_df.pickle","rb"))
checkdf.head()

,track_id,track_name,track_artist,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,playlist_genre,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,text,BERT_embeddings
0,6f807x0ima9a1j3VPbc7VN,I Don't Care (with Justin Bieber) - Loud Luxur...,Ed Sheeran,66,2oCs0DGTsRO98Gh5ZSl2Cx,I Don't Care (with Justin Bieber) [Loud Luxury...,2019-06-14,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,1,0.0583,0.1020,0.000000,0.0653,0.518,122.036,194754,I Don't Care (with Justin Bieber) - Loud Luxur...,"[tensor(0.1147), tensor(-0.8324), tensor(0.958..."
1,0r7CVbZTWZgbTCYdfa2P31,Memories - Dillon Francis Remix,Maroon 5,67,63rPSO264uRjW1X5E6cWv6,Memories (Dillon Francis Remix),2019-12-13,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,1,0.0373,0.0724,0.004210,0.3570,0.693,99.972,162600,Memories - Dillon Francis Remix is a song by a...,"[tensor(0.1133), tensor(-0.8689), tensor(1.062..."
2,1z1Hg7Vb0AhHDiEmnDE79l,All the Time - Don Diablo Remix,Zara Larsson,70,1HoSmj2eLcsrR0vE9gThr4,All the Time (Don Diablo Remix),2019-07-05,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,0,0.0742,0.0794,0.000023,0.1100,0.613,124.008,176616,All the Time - Don Diablo Remix is a song by a...,"[tensor(-0.1786), tensor(-0.8082), tensor(0.98..."
3,75FpbthrwQmzHlBJLuGdC7,Call You Mine - Keanu Silva Remix,The Chainsmokers,60,1nqYsOef1yKKuGOVchbsk6,Call You Mine - The Remixes,2019-07-19,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,1,0.1020,0.0287,0.000009,0.2040,0.277,121.956,169093,Call You Mine - Keanu Silva Remix is a song by...,"[tensor(0.0896), tensor(-0.7529), tensor(0.857..."
4,1e8PAfcKUYoKkxPhrHqw4x,Someone You Loved - Future Humans Remix,Lewis Capaldi,69,7m7vv9wlQ4i0LFuJiE2zsQ,Someone You Loved (Future Humans Remix),2019-03-05,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,1,0.0359,0.0803,0.000000,0.0833,0.725,123.976,189052,Someone You Loved - Future Humans Remix is a s...,"[tensor(-0.0501), tensor(-0.7875), tensor(1.08..."


In [2]:
checkdf["BERT_embeddings"][0]

tensor([ 1.1472e-01, -8.3238e-01,  9.5877e-01,  7.0818e-01,  3.8216e-01,
        -3.4943e-01,  1.5992e-01, -7.8259e-01,  8.2220e-02, -5.8277e-01,
        -3.6338e-01, -1.9349e-01, -1.3425e+00,  1.5406e-01, -4.4850e-01,
         1.5707e-01,  3.1937e-01, -1.1188e+00,  1.5519e-01,  2.3950e-01,
        -1.5006e-01, -6.6432e-03,  6.8278e-01,  5.2417e-01, -3.4914e-01,
         1.3505e-01, -2.4124e-01,  8.2486e-02,  1.6451e-02, -7.2776e-01,
         2.3853e-01, -2.3035e-01, -2.7187e-01, -4.9919e-01, -4.0014e-02,
        -1.1152e-01, -4.6274e-02, -8.1965e-01, -9.1994e-02, -1.0485e-01,
         1.7057e-01,  1.5077e-01, -3.0182e-01, -7.7300e-01, -1.5164e-01,
         3.0532e-01, -3.2515e+00,  5.7457e-01,  1.3903e-01, -4.8841e-01,
         6.9226e-01,  3.4927e-01, -1.1804e+00,  1.0294e-01,  1.1707e-01,
        -2.5709e-02, -3.6140e-01, -1.1920e-01, -8.6730e-01, -2.8727e-01,
         4.2365e-01, -1.6379e-03,  5.9392e-01, -2.7858e-01,  8.7196e-02,
        -7.8184e-01, -4.6835e-01, -5.1875e-01, -5.7

In [14]:
px = pd.DataFrame(checkdf["BERT_embeddings"][0].unsqueeze(dim = 0)).astype("float")
for i in range(1,len(checkdf)):
    if i % 100 == 0:
        print(i)
    temp_df = pd.DataFrame(checkdf["BERT_embeddings"][i].unsqueeze(dim = 0)).astype("float")
    px = pd.concat([px, temp_df],ignore_index = True)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
1850

In [16]:
px.head()

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.114718,-0.832378,0.958771,0.708180,0.382164,-0.349433,0.159918,-0.782590,0.082220,-0.582768,...,0.474889,0.294219,0.516970,-0.750061,0.351938,0.357721,-0.068786,0.161389,-0.447223,0.099216
1,0.113306,-0.868864,1.062681,0.722131,0.204493,-0.217453,0.319892,-0.562883,0.072262,-0.592896,...,0.504099,0.268597,0.744696,-0.577474,0.501883,0.382588,-0.095205,0.156200,-0.523669,0.115569
2,-0.178616,-0.808245,0.983175,0.641901,0.025184,-0.257694,0.338703,-0.581231,0.054863,-0.598674,...,0.240050,0.091955,0.748596,-0.494741,0.343062,0.358682,-0.269875,0.032701,-0.322348,0.361017
3,0.089576,-0.752860,0.857660,0.630942,0.254457,-0.238870,0.137968,-0.689916,0.170709,-0.441062,...,0.490758,0.201577,0.728372,-0.667164,0.429742,0.540678,0.003768,0.149373,-0.251839,0.301827
4,-0.050071,-0.787461,1.085825,0.612994,0.199960,-0.114725,0.193144,-0.576667,0.079001,-0.475714,...,0.420919,0.207009,0.671797,-0.631807,0.505278,0.328273,-0.106546,0.056408,-0.348282,0.337523


In [18]:
px.to_csv("BERT_embeddings_dataframe.csv")